In [1]:
import logging
import sys


import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint
from cicada.logging import Logger

logging.basicConfig(level=logging.INFO)

def main(communicator):
    log = Logger(logging.getLogger(), communicator)
    protocol = AdditiveProtocolSuite(communicator, order=251, encoding=FixedPoint(precision=4))
    generator = numpy.random.default_rng(seed=1234)

    for i in range(10):

        log.info("*" * 40, src=0)
        
        secret_share = protocol.field_uniform(generator=generator)
        secret = protocol.reveal(secret_share)
        log.info(f"Player {communicator.rank} secret: {secret}")

        floor_share = protocol.floor(secret_share)
        floor = protocol.reveal(floor_share)
        log.info(f"Player {communicator.rank} floor: {floor}")


SocketCommunicator.run(world_size=3, fn=main);

INFO:root:****************************************
INFO:root:Player 0 secret: 3.5
INFO:root:Player 1 secret: 3.5
INFO:root:Player 2 secret: 3.5
INFO:root:Player 0 floor: 3.0
INFO:root:Player 1 floor: 3.0
INFO:root:Player 2 floor: 3.0
INFO:root:****************************************
INFO:root:Player 0 secret: -1.5
INFO:root:Player 1 secret: -1.5
INFO:root:Player 2 secret: -1.5
INFO:root:Player 0 floor: -2.0
INFO:root:Player 1 floor: -2.0
INFO:root:Player 2 floor: -2.0
INFO:root:****************************************
INFO:root:Player 0 secret: -7.1875
INFO:root:Player 1 secret: -7.1875
INFO:root:Player 2 secret: -7.1875
INFO:root:Player 0 floor: 7.6875
INFO:root:Player 1 floor: 7.6875
INFO:root:Player 2 floor: 7.6875
INFO:root:****************************************
INFO:root:Player 0 secret: -0.75
INFO:root:Player 1 secret: -0.75
INFO:root:Player 2 secret: -0.75
INFO:root:Player 0 floor: -1.0
INFO:root:Player 1 floor: -1.0
INFO:root:Player 2 floor: -1.0
INFO:root:******************